## GRAY IMAGES

In [ ]:
from PIL import Image
import pandas as pd
import os

In [ ]:
anot_path = "./data/attention.csv"
images_path = "./data/datasets/"

gray_images_path = "./data/grayimage"

In [ ]:
df = pd.read_csv(anot_path)
df.columns = ['filename', 'score']
df['score'].value_counts()

In [ ]:
save_path = "data/gray_data.csv"

data = {
    'width': [],
    'height': [],
    'file_path': [],
    'score': []
}

if not os.path.exists(gray_images_path):
    os.makedirs(gray_images_path)

for item, score in zip(df['filename'].tolist(), df['score'].tolist()):
    file_path = os.path.join(images_path, item)
    gray_path = os.path.join(gray_images_path, item)

    img = Image.open(file_path).convert("L")
    
    img.save(gray_path)
    
    data['width'].append(img.size[0])
    data['height'].append(img.size[1])
    data['file_path'].append(gray_path)
    data['score'].append(score)

In [ ]:
pd.DataFrame(data).to_csv(save_path, index=False)

## BLUR CANNY CONTOUR 

In [ ]:
import pandas as pd
from PIL import Image
import random
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import torch

# anomali data 69, 5, 36
# rd_idx = 5

file_path = "data/gray_data.csv"
df = pd.read_csv(file_path)

def Proccess(df = df, idx = None, biar_rapih = False):
    if idx == None:
        idx = random.randint(0, len(df['file_path'].tolist()))
    
    sample_image_name = df['file_path'].tolist()[idx]
    score = df['score'].tolist()[idx]

    img_np = np.array(Image.open(sample_image_name))
    
    if not biar_rapih:
        # gausah didebug ini [biar apa biarin | pusing gw njir ngeliatnya]
        print(f"image idx: {idx} | score: {score} | filename: {sample_image_name}")
        print(f"image size: {img_np.shape}")

    blank = np.zeros(shape=img_np.shape, dtype='uint8')

    gaussBlur = cv.GaussianBlur(img_np, (175, 175), 0.3)
    canny = cv.Canny(gaussBlur, 100, 100)

    _, tresh = cv.threshold(img_np, 125, 255, cv.THRESH_BINARY)
    _, blurtresh = cv.threshold(gaussBlur, 125, 255, cv.THRESH_BINARY)

    contours, _ = cv.findContours(canny, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    cv.drawContours(blank, contours, -1, 255, 1)
    
    images = [img_np, gaussBlur, canny, tresh, blank]
    titles = ['Gray', 'Gaussian Blur', 'Canny', 'Threshold', 'Contours Drawn']
    
    return images, titles

def ShowImage(images, titles):
    
    plt.figure(figsize=(15, 8))

    for i in range(len(images)):
        plt.subplot(2, 3, i + 1)  # 2 baris, 3 kolom
        if len(images[i].shape) == 2:  # grayscale
            plt.imshow(images[i], cmap='gray')
        else:  # RGB image
            plt.imshow(cv.cvtColor(images[i], cv.COLOR_BGR2RGB))
        plt.title(titles[i])
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
max_w = max(df['width'].tolist())
min_w = min(df['width'].tolist())

max_h = max(df['height'].tolist())
min_h = min(df['height'].tolist())

print(f"max-width: {max_w} | min-width: {min_w} | max-height: {max_h} | min-height: {min_h}")

In [ ]:
for i in range(3):
    images, titles = Proccess()
    ShowImage(images, titles)

## LOAD DATA INTO NDARRAY

In [ ]:
import pandas as pd
import numpy as np
import cv2 as cv

file_path = "data/gray_data.csv"
df = pd.read_csv(file_path)

df.describe()

In [ ]:
def resize(np_img, w = 512, h = 256):
    return cv.resize(np_img, dsize=(w, h), interpolation=cv.INTER_LINEAR)

In [ ]:
gray_np = [] # gray images
gaussBlur_np = []
canny_np = []
tresh_np = []
contour_np = []

# load to array
for i in range(len(df['file_path'].tolist())):
    '''
    images = [
        img_np, gaussBlur, canny, tresh, blank]
        [gray, gaussblur(gray), canny(gray), tresh(gray), contour(grey)
    ]
    '''
    
    images, _ = Proccess(idx=i, biar_rapih=True)
    # print(type(resize(images[0])))
    # print(resize(images[0]))
    # print(resize(images[0]).shape)
    # break
    gray_np.append(resize(images[0]))
    gaussBlur_np.append(resize(images[1]))
    canny_np.append(resize(images[2]))
    tresh_np.append(resize(images[3]))
    contour_np.append(resize(images[4]))

# transform into 
gray_np = np.array(gray_np)
gaussBlur_np = np.array(gaussBlur_np)
canny_np = np.array(canny_np)
tresh_np = np.array(tresh_np)
contour_np = np.array(contour_np)

In [ ]:
print(gray_np.shape, gaussBlur_np.shape, canny_np.shape, tresh_np.shape, contour_np.shape)

# pake dah mau pake yang mana

In [ ]:
X = gray_np.reshape(gray_np.shape[0], -1)
X = X / 255
y = np.array(df['score']).reshape(-1, 1)

X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
type(X_train[0]), X_train[0]

In [ ]:
type(y_train[0]), y_train[0]

In [ ]:
print(y.shape[0])

In [ ]:
X = gray_np.reshape(-1, 1, 512, 256).astype(np.float32)
X = X / 255.0
y = np.argmax(y, axis=1)

# convert data to tensor
X = torch.tensor(X)
y = torch.tensor(y, dtype=torch.long)

X.shape, y.shape

In [ ]:
torch.tensor(y[0]).shape

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X, y)
trainloader = DataLoader(dataset, batch_size=32, shuffle=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

## EFFICIENT NET B0

In [ ]:
!nvidia-smi

In [ ]:
from os import listdir
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc
import cv2
import os
import torch
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
import torch.nn as nn
import json
from datetime import datetime
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
model = models.efficientnet_b7()

conv1_weight = model.features[0][0].weight.data
new_conv1_weight = conv1_weight.mean(dim=1, keepdim=True)

model.features[0][0] = torch.nn.Conv2d(3, 32, kernel_size=(3,3), stride=(2,2), padding=(1,1), bias=False)
model.features[0][0].weight.data = new_conv1_weight

x = torch.randn(1,1, 512,256)

for param in model.parameters():
    param.requires_grad = False

print(model)

In [ ]:
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 5)

print(model)

In [ ]:
num_classes = 5
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5, inplace=True),
    nn.Linear(model.classifier[1].in_features, num_classes)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)

In [ ]:
model.to(device)

In [ ]:
name_model = 'EfficientNet_B7'

In [ ]:
batch_size = 32

for batch_idx, (inputs, targets) in enumerate(trainloader):
    print(f"Batch {batch_idx+1}")
    print(f"Inputs Shape : {inputs.shape}, Targets Shape : {targets.shape}")
    break

In [ ]:
from torch.utils.data import random_split

dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, shuffle=True)
test_loader = DataLoader(test_dataset, shuffle=False)

for input, target in train_loader:
    print(input.shape)
    print(target.shape)
    break

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()

    running_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for inputs, targets, in train_loader:
        inputs,targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs,targets)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == targets).sum().item()
        total_preds += targets.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct_preds / total_preds

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss : {epoch_loss:.4f}, Accuracy : {epoch_accuracy:.3f}")

## Testing

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = 100 * correct / total
print(f"Akurasi pada data test : {accuracy:.3f}%")
print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds))

In [ ]:
import matplotlib.pyplot as plt

wrong = 0
for inputs, labels in test_loader:
    outputs = model(inputs.to(device))
    _, preds = torch.max(outputs, 1)
    for i in range(len(preds)):
        if preds[i] != labels[i]:
            plt.imshow(inputs[i].squeeze(), cmap="gray")
            plt.title(f"True: {labels[i]}, Pred: {preds[i]}")
            plt.show()
            wrong += 1
        if wrong > 5:
            break
    if wrong > 5:
        break